In [20]:

import itertools
import matplotlib as mpl
import numpy as np
import os
import tensorflow as tf
import tensorflow.contrib.slim as slim
import time
import seaborn as sns

import sklearn as sk

from matplotlib import pyplot as plt
from scipy.misc import imsave
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import warnings , os
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


## https://github.com/nakumgaurav/Anomaly-Detection_Varitaional-Autoencoders

### [tf.flags](https://daeson.tistory.com/256)
* int ,float , boolean , string 값 저장해서 쉽게 사용 할 수 있게 해주는 것!


In [21]:
sns.set_style('whitegrid')

distributions = tf.distributions

####Delete all flags before declare#####
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:    
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)
tf.reset_default_graph()

flags = tf.app.flags
## 현재 디렉토리를 만드는 듯?
flags.DEFINE_string('data_dir', 'data/', 'Directory for data')
flags.DEFINE_string('logdir', '/home/advice/Python/SR/board/VAE_OD', 'Directory for logs')


In [22]:
flags.DEFINE_integer('latent_dim', 2, 'Latent dimensionality of model')
flags.DEFINE_integer('batch_size', 256 , 'Minibatch size')
flags.DEFINE_integer('visual_size', 1000 , 'visualization size')
flags.DEFINE_integer('n_samples', 1, 'Number of samples to save')
flags.DEFINE_integer('print_every', 100, 'Print every n iterations')
flags.DEFINE_integer('hidden_size', 200, 'Hidden size for neural networks')
flags.DEFINE_integer('n_iterations', 1000, 'number of iterations')
flags.DEFINE_float('threshold', 7.0, 'Threshold for recon probab')

FLAGS = flags.FLAGS

In [23]:
FLAGS._flags() 

{'data_dir': <absl.flags._flag.Flag at 0x7f8363daada0>,
 'logdir': <absl.flags._flag.Flag at 0x7f8363daa940>,
 'latent_dim': <absl.flags._flag.Flag at 0x7f8363dc8240>,
 'batch_size': <absl.flags._flag.Flag at 0x7f8363dc89b0>,
 'visual_size': <absl.flags._flag.Flag at 0x7f8363dc81d0>,
 'n_samples': <absl.flags._flag.Flag at 0x7f8363dc8048>,
 'print_every': <absl.flags._flag.Flag at 0x7f8363dc8780>,
 'hidden_size': <absl.flags._flag.Flag at 0x7f7f18da2f98>,
 'n_iterations': <absl.flags._flag.Flag at 0x7f7f18da2cc0>,
 'threshold': <absl.flags._flag.Flag at 0x7f7f18da2780>}

In [24]:
def inference_network(x, latent_dim, hidden_size , p):
    """Construct an inference network parametrizing a Gaussian.
        Args:
        x: A batch of MNIST digits.
        latent_dim: The latent dimensionality.
        hidden_size: The size of the neural net hidden layers.

      Returns:
        mu: Mean parameters for the variational family Normal
        sigma: Standard deviation parameters for the variational family Normal
     """
    with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.leaky_relu):
        net = slim.flatten(x)
        net = slim.fully_connected(net, hidden_size )
        net = slim.fully_connected(net, hidden_size )
#         net = tf.contrib.layers.batch_norm(net, center=True, scale=True, 
#                                           is_training= p ,
#                                           scope='generative_bn')
#         net = slim.fully_connected(net, hidden_size )
#         net = tf.contrib.layers.batch_norm(net, center=True, scale=True , 
#                                           is_training= p ,
#                                           scope='generative_bn2')
        net = slim.fully_connected(net, hidden_size  )
        gaussian_params = slim.fully_connected(net, latent_dim * 2, activation_fn=None)
  # The mean parameter is unconstrained
    mu = gaussian_params[:, :latent_dim]
  # The standard deviation must be positive. Parametrize with a softplus
    sigma = tf.nn.softplus(gaussian_params[:, latent_dim:])
    return mu, sigma

In [25]:
def generative_network(z, hidden_size , p):
    """Build a generative network parametrizing the likelihood of the data

  Args:
    z: Samples of latent variables
    hidden_size: Size of the hidden state of the neural net

  Returns:
    bernoulli_logits: logits for the Bernoulli likelihood of the data
  """
    with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.leaky_relu):
        net = slim.fully_connected(z, hidden_size )
        #net = tf.nn.dropout(net, p )
#         net = tf.contrib.layers.batch_norm(net, center=True, scale=True, 
#                                           is_training=p,
#                                           scope='inference_bn')
#         net = slim.fully_connected(net, hidden_size )
#         net = tf.contrib.layers.batch_norm(net, center=True, scale=True, 
#                                           is_training= p ,
#                                           scope='inference_bn2')
        net = slim.fully_connected(net, hidden_size )
        net = slim.fully_connected(net, hidden_size )
        net = slim.fully_connected(net, hidden_size )
        
        bernoulli_logits = slim.fully_connected(net, 784, activation_fn=None)
        bernoulli_logits = tf.reshape(bernoulli_logits, [-1, 28, 28, 1])
        
    return bernoulli_logits

In [26]:
def train():
  # Train a Variational Autoencoder on MNIST

  # Input placeholders
    prob = tf.placeholder(tf.bool, name ="bn_bool")
    with tf.name_scope('data'):
        x = tf.placeholder(tf.float32, [None, 28, 28, 1])
        tf.summary.image('data', x)
    with tf.variable_scope('variational'):
        q_mu, q_sigma = inference_network(x=x,
                                          latent_dim=FLAGS.latent_dim,
                                          hidden_size=FLAGS.hidden_size , 
                                          p = prob)
        
    # The variational distribution is a Normal with mean and standard
    # deviation given by the inference network
    q_z = distributions.Normal(loc=q_mu, scale=q_sigma)
    assert q_z.reparameterization_type == distributions.FULLY_REPARAMETERIZED
    
    with tf.variable_scope('model'):
    # The likelihood is Bernoulli-distributed with logits given by the
    # generative network
        mySample = q_z.sample()
        p_x_given_z_logits = generative_network(z=mySample,
                                                hidden_size=FLAGS.hidden_size , 
                                                p = prob)
        p_x_given_z = distributions.Bernoulli(logits=p_x_given_z_logits)
        posterior_predictive_samples = p_x_given_z.sample()
        tf.summary.image('posterior_predictive',
                         tf.cast(posterior_predictive_samples, tf.float32))

  # Take samples from the prior
    with tf.variable_scope('model', reuse=True):
        p_z = distributions.Normal(loc=np.zeros(FLAGS.latent_dim, dtype=np.float32),
                                   scale=np.ones(FLAGS.latent_dim, dtype=np.float32))
        p_z_sample = p_z.sample(FLAGS.n_samples)
        p_x_given_z_logits = generative_network(z=p_z_sample,
                                                hidden_size=FLAGS.hidden_size , 
                                                p = prob)
    prior_predictive = distributions.Bernoulli(logits=p_x_given_z_logits)
    prior_predictive_samples = prior_predictive.sample()
    tf.summary.image('prior_predictive',
                     tf.cast(prior_predictive_samples, tf.float32))

#   # Take samples from the prior with a placeholder
    with tf.variable_scope('model', reuse=True):
        z_input = tf.placeholder(tf.float32, [None, FLAGS.latent_dim])
        p_x_given_z_logits = generative_network(z=z_input,
                                                hidden_size=FLAGS.hidden_size , 
                                                p = prob)
        prior_predictive_inp = distributions.Bernoulli(logits=p_x_given_z_logits)
        prior_predictive_inp_sample = prior_predictive_inp.sample()

  # Build the evidence lower bound (ELBO) or the negative loss
    kl = tf.reduce_sum(distributions.kl_divergence(q_z, p_z), 1)
#   tense = p_x_given_z.log_prob(x)
    expected_log_likelihood = tf.reduce_sum(p_x_given_z.log_prob(x),
                                          [1, 2, 3])
    #expected_log_likelihood = tf.reduce_sum(p_x_given_z.log_prob(x),[1, 2, 3])
  # print "p_x_given_z.log_prob(x).shape", p_x_given_z.log_prob(x).shape
  # shapeTensor =  tf.shape(p_x_given_z.log_prob(x))

    #elbo = -tf.reduce_sum(expected_log_likelihood - kl, 0)
     
    #marginal_likelihood = tf.reduce_mean(expected_log_likelihood)
    #KL_divergence = tf.reduce_mean(kl)
    #elbo = - (marginal_likelihood - KL_divergence ) 
    elbo = -tf.reduce_sum(expected_log_likelihood - kl, 0)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize( elbo )
    summary_op = tf.summary.merge_all() ## # Merge all the summaries
    init_op = tf.global_variables_initializer()
  # Run training
    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    sess.run(init_op)
    save_file = "./../../Save_Model/VAE_OD.meta"
    try :
        saver = tf.train.import_meta_graph(save_file)
        saver.restore(sess, tf.train.latest_checkpoint('./../../Save_Model/'))
    except Exception as e :
        pass
    mnist = read_data_sets(FLAGS.data_dir, one_hot=True)
    train, test = mnist.train, mnist.test
    trainX, trainY, testX, testY = train.images, train.labels, test.images, test.labels
    ### 0만??? 
    k = 0
    print(trainY[0:5])
    mask = trainY[:,k]==0 # omit all examples of the class k  0을 제외하고 1~9만 뽑기
    print(mask[0:5])
    trainX = trainX[mask]
    trainY = trainY[mask]
    n = trainX.shape[0]
    print('Saving TensorBoard summaries and images to: %s' % FLAGS.logdir)
    train_writer = tf.summary.FileWriter(FLAGS.logdir, sess.graph)
  # Get fixed MNIST digits for plotting posterior means during training
    t0 = time.time()
    cut_line = 1.0 * 80 # FLAGS.batch_size
    for i in range(FLAGS.n_iterations):
        batch_idx = np.arange(len(trainX))
        np.random.shuffle(batch_idx)
        if i % 100 == 0 :
            print("Epoch : ", i)
        for n in range(0, ( len(trainX) // FLAGS.batch_size) * FLAGS.batch_size, FLAGS.batch_size) :
            batch = batch_idx[n: n + FLAGS.batch_size]
        # Re-binarize the data at every batch; this improves results
#     np_x, _ = train.next_batch(FLAGS.batch_size)
            np_x = trainX[batch,:]
            np_y = trainY[batch,:]
    #     print(np_x.shape)
    #     np_x += np.random.randn(np_x.shape)
            np_x = np_x.reshape(FLAGS.batch_size, 28, 28, 1)
            np_x = (np_x > 0.5).astype(np.float32)
        #     print np_x.shape
            sess.run(train_op, {x: np_x , prob : 1 })
#     print samples.shape
    # Print progress and save samples every so often
        if i % FLAGS.print_every == 0:
            np_elbo, summary_str = sess.run([elbo, summary_op], {x: np_x , prob : 0})
            train_writer.add_summary(summary_str, i)
            print('Iteration: {0:d} ELBO: {1:.3f} s/iter: {2:.3e}'.format(
                i, np_elbo / FLAGS.batch_size, (time.time() - t0) / FLAGS.print_every))
            if np_elbo / FLAGS.batch_size < cut_line : 
                print(" {} -> {}".format(cut_line , np_elbo / FLAGS.batch_size ))
                saver.save(sess, './../../Save_Model/VAE_OD')
                cut_line = np_elbo / FLAGS.batch_size
                
            t0 = time.time() 
          # Save samples
            np_posterior_samples, np_prior_samples = sess.run(
                [posterior_predictive_samples, prior_predictive_samples], {x: np_x , prob : 0 })
            for k in range(FLAGS.n_samples):
                f_name = os.path.join(
                    FLAGS.logdir, 'iter_%d_posterior_predictive_%d_data.jpg' % (i, k))
                imsave(f_name, np_x[k, :, :, 0])
                f_name = os.path.join(
                    FLAGS.logdir, 'iter_%d_posterior_predictive_%d_sample.jpg' % (i, k))
                imsave(f_name, np_posterior_samples[k, :, :, 0])
                f_name = os.path.join(
                    FLAGS.logdir, 'iter_%d_prior_predictive_%d.jpg' % (i, k))
                imsave(f_name, np_prior_samples[k, :, :, 0])

          # Plot the posterior predictive space
            if FLAGS.latent_dim == 2:
                visual_n = np.floor(np.random.rand(FLAGS.visual_size)*n).astype(int)
                vis_x = trainX[visual_n,:]
                vis_y = trainY[visual_n,:]
        #     print(np_x.shape)
        #     np_x += np.random.randn(np_x.shape)
                vis_x = vis_x.reshape(FLAGS.visual_size, 28, 28, 1)
                vis_x = (vis_x > 0.5).astype(np.float32)
                np_q_mu = sess.run(q_mu, {x: vis_x , prob : 0 })
                cmap = mpl.colors.ListedColormap(sns.color_palette("husl" , 10))
                f, ax = plt.subplots(1, figsize=(6 * 1.1618, 6))
                print(np.shape(np_q_mu))
                print(np.shape(vis_y))
                im = ax.scatter(np_q_mu[:, 0], np_q_mu[:, 1], c=np.argmax(vis_y, 1),
                                cmap=cmap, alpha=0.7)
                ax.set_xlabel('First dimension of sampled latent variable $z_1$')
                ax.set_ylabel('Second dimension of sampled latent variable mean $z_2$')
                ax.set_xlim([-10., 10.])
                ax.set_ylim([-10., 10.])
                f.colorbar(im, ax=ax, label='Digit class')
                plt.tight_layout()
                plt.savefig(os.path.join(FLAGS.logdir,
                                         'posterior_predictive_map_frame_%d.png' % i))
                plt.close()

                nx = ny = 20
                x_values = np.linspace(-3, 3, nx)
                y_values = np.linspace(-3, 3, ny)
                canvas = np.empty((28 * ny, 28 * nx))
                for ii, yi in enumerate(x_values):
                    for j, xi in enumerate(y_values):
                        np_z = np.array([[xi, yi]])
                        x_mean = sess.run(prior_predictive_inp_sample, {z_input: np_z , prob : 0})
                        canvas[(nx - ii - 1) * 28:(nx - ii) * 28, j *
                           28:(j + 1) * 28] = x_mean[0].reshape(28, 28)
                imsave(os.path.join(FLAGS.logdir,'prior_predictive_map_frame_%d.png' % i), canvas)
#                 plt.figure(figsize=(8, 10))
#                 Xi, Yi = np.meshgrid(x_values, y_values)
#                 plt.imshow(canvas, origin="upper")
#                 plt.tight_layout()
                #plt.savefig()
    # Make the gifs 다 학습시키고 gif 만들기?
    if FLAGS.latent_dim == 2:
        print("Image -> Gif")
        os.system('convert -delay 15 -loop 0 {0}/posterior_predictive_map_frame*png {0}/posterior_predictive.gif'
                  .format(FLAGS.logdir))
        os.system('convert -delay 15 -loop 0 {0}/prior_predictive_map_frame*png {0}/prior_predictive.gif'
                  .format(FLAGS.logdir))
    ## 이놈은 그냥 샘플 뽑기 
    np_x_fixed, np_y = test.next_batch(5000)
    np_x_fixed = np_x_fixed.reshape(5000, 28, 28, 1)
    np_x_fixed = (np_x_fixed > 0.5).astype(np.float32)
    ## class가 0인놈만 뽑기 (outlier같은 느낌이 되는거겠지?) 
    np_y = np.asarray(np_y[:,k]==1, dtype=np.float32)
  # evaluate the test samples 특정 기준으로만?!
    np_posterior_samples, np_prior_samples = sess.run(
        [posterior_predictive_samples, prior_predictive_samples], {x: np_x_fixed  , prob : 0 })
    ## 이것의 의미는 뭘까? 
    norm_vec = np.linalg.norm((np_x_fixed - np_posterior_samples).reshape(5000,784) , axis=1)
#   np_prior_samples[k, :, :, 0] 
    ## 특정 threshold 보다 크면 TRUE라고 내뱉기
    ## 학습시에는 0을 뺴고 한 상태 여기는 0이 나올 수도 있는 상태
    ## 그러므로 만약 이 놈이 0을 잘 탐지했다면 norm_vec 이 크게 나올 것이다. 
    anomaly_vector = np.asarray(norm_vec > FLAGS.threshold, np.float32)
    ## mse가 커서 outlier라고 예측
    y_pred = anomaly_vector
    ## 실제 타켓에서 0인 것에 대한 T/F
    y_real = np_y
    val_accuracy = np.sum(y_pred == y_real)*1.0/5000.0
#   y_truth = tf.placeholder(tf.float32, [5000,])
#   corr_pred = tf.equal(y_p, y_truth)
#   accuracy = tf.reduce_mean(tf.cast(corr_pred, "float"))
#   val_accuracy, y_pred, rp = sess.run([accuracy, y_p, expected_log_likelihood], feed_dict={x:np_x_fixed, y_truth:np_y})
    outlier_idx = np_y == 1
    normal_idx = np_y == 0
    normal_data = norm_vec[normal_idx]
    outlier_data = norm_vec[outlier_idx] 
    plt.hist(normal_data, label= "Normal")
    plt.hist(outlier_data, label= "Outlier")
    plt.legend()
    plt.show()
    print("validation accuracy:", val_accuracy)
#   y_true = np.asarray(np.argmax(np_y,1), np.float32)
#   print y_real, y_pred
    print(np.sum(y_pred)) ## 이상하다고 말하는 놈들의 수
    print(np.sum(y_real)) ## 실제 정상의 수 
    print("Precision", sk.metrics.precision_score(y_real, y_pred))
    print("Recall", sk.metrics.recall_score(y_real, y_pred))
    print("f1_score", sk.metrics.f1_score(y_real, y_pred))
    print("confusion_matrix")
    print(sk.metrics.confusion_matrix(y_real, y_pred)  )
    fpr, tpr, thresholds = sk.metrics.roc_curve(y_real, y_pred)
    auc = sk.metrics.roc_auc_score(y_real, y_pred)
    print("{} , AUC={:.3f}".format(i , auc))
    
   

In [27]:
def main(_):
    if tf.gfile.Exists(FLAGS.logdir):
        ## 전부다 제거하기
        tf.gfile.DeleteRecursively(FLAGS.logdir)
        ## 폴더 새로 생성하는 것 같음
        tf.gfile.MakeDirs(FLAGS.logdir)
        train()

In [ ]:
if __name__ == '__main__':
    tf.reset_default_graph()
    tf.app.run()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[ True  True  True  True  True]
Saving TensorBoard summaries and images to: /home/advice/Python/SR/board/VAE_OD
Epoch :  0
Iteration: 0 ELBO: 156.962 s/iter: 1.612e-02
(1000, 2)
(1000, 10)
Epoch :  100
Iteration: 100 ELBO: 127.894 s/iter: 1.227e+00
(1000, 2)
(1000, 10)
Epoch :  200
Iteration: 200 ELBO: 120.930 s/iter: 1.217e+00
(1000, 2)
(1000, 10)
Epoch :  300
Iteration: 300 ELBO: 119.005 s/iter: 1.229e+00
(1000, 2)
(1000, 10)
Epoch :  400
Iteration: 400 ELBO: 121.274 s/iter: 1.229e+00
(1000, 2)
(1000, 10)
Epoch :  500
Iteration: 500 ELBO: 115.848 s/iter: 1.212e+00
(1000, 2)
(1000, 10)
